In [13]:
# 00 ) Packages
import pandas as pd
import numpy as np
import xlrd
import os
import pprint
from tkinter import filedialog as fd
os.chdir(r"C:\Users\pallist\Desktop\ΤΡΕΧΟΝΤΑ\testing folder")
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
# pd.options.display.max_rows = 999
pd.options.mode.chained_assignment = None 

In [14]:
# 0 ) Importing Files
onomata = pd.read_excel("ΒΑΣΗ ΟΝΟΜΑΤΩΝ (ΑΡΣΕΝΙΚΑ).xlsx")
lista_onomaton = onomata.iloc[:,1]
file= fd.askopenfilename()
stoixeia = pd.read_excel(file)

In [15]:
# 1) Insert columns of raw data into a new dataframe (if there is extra informtion about the person, add those columns too)

test = pd.DataFrame()
test["Ton"] = ""
test["Surname"] = stoixeia.iloc[:,0]
test["Name"] = stoixeia.iloc[:,1]
test["Father"] = stoixeia.iloc[:,2]

if len(stoixeia.columns) > 3 and not stoixeia.iloc[:,3].str.isnumeric().all():
    test["City"] = stoixeia.iloc[:,3]
    test["Address"] = stoixeia.iloc[:,4].astype(str)
    test["Number"] = stoixeia.iloc[:,5]
    test["Number"] = test["Number"].astype(str)
    if  is_numeric_dtype(stoixeia.iloc[:,6]):
        test["TK"] = stoixeia.iloc[:,6].fillna(0).astype(int).astype(str)
    else :
        test["TK"]  = ""
    if is_numeric_dtype(stoixeia.iloc[:,7]):
        test["ΑΦΜ"] = stoixeia.iloc[:,7].fillna(0).astype(int).astype(str)
    else :
        test["ΑΦΜ"]  = ""    
    test["stoixeia"] = ""
test.columns

Index(['Ton', 'Surname', 'Name', 'Father', 'City', 'Address', 'Number', 'TK',
       'ΑΦΜ', 'stoixeia'],
      dtype='object')

In [16]:
# 2) Create hashmap dictionary για τις εξαιρέσεις των άρθρων και πόλεων 


ton_list = ["ΜΙΧΑΗΛ", "ΕΜΜΑΝΟΥΗΛ", "ΣΠΥΡΙΔΩΝ", "ΑΧΙΛΛΕΥΣ", "ΟΔΥΣΣΕΥΣ"]



keys = ["ΧΑΝΙΑ", "ΦΑΡΣΑΛΑ", "ΙΩΑΝΝΙΝΑ", "ΣΕΠΟΛΙΑ", "ΤΡΙΚΑΛΑ","ΑΝΩ ΛΙΟΣΙΑ", "ΚΑΛΥΒΙΑ", "ΜΕΓΑΡΑ","ΝΈΟ ΗΡΑΚΛΕΙΟ", "ΠΑΛΑΙΟ ΦΑΛΗΡΟ", "ΝΕΑ ΙΩΝΙΑ", "ΑΓΙΟΙ ΑΝΑΡΓΥΡΟΙ", "ΑΓΙΑ ΠΑΡΑΣΚΕΥΗ"]

values = ["ΧΑΝΙΩΝ", "ΦΑΡΣΑΛΩΝ", "ΙΩΑΝΝΙΝΩΝ", "ΣΕΠΟΛΙΩΝ","ΤΡΙΚΑΛΩΝ", "ΑΝΩ ΛΙΟΣΙΩΝ", "ΚΑΛΥΒΙΩΝ", "ΜΕΓΑΡΩΝ", "ΝΕΟΥ ΗΡΑΚΛΕΙΟΥ", "ΠΑΛΑΙΟΥ ΦΑΛΗΡΟΥ", "ΝΕΑΣ ΙΩΝΙΑΣ" , "ΑΓΙΩΝ ΑΝΑΡΓΥΡΩΝ", "ΑΓΙΑΣ ΠΑΡΑΣΚΕΥΗΣ"]

city_dict = dict(zip(keys, values))

In [17]:
#  3) Ονόματα

def names () :
    ton_list = []
    for t in test["Name"] :
            x = str(t)
            for ton1 in lista_onomaton :
                if x == ton1:
                    ton_list.append("Τον")            
            if x[-2:] == "ΟΣ" or x[-2:] == "ΗΣ" :
                ton_list.append("Τον")
            elif x[-1:] == "Α" or x[-1:] == "Η":
                ton_list.append("Την")
            else :
                ton_list.append("Την")    
    test["Ton"] = ton_list

    surname_list = []
    
    
    for t in test["Surname"] :
        x = str(t)
        if x[-1] == "Σ" :
            x = x[:-1]
        surname_list.append(t)        
    test["Surname"] = surname_list        

    name_list = []

    for t in test["Name"] :
        x = str(t)
        if x[-1] == "Σ" :
            x = t[:-1]
        name_list.append(t)

    test["Name"] = name_list

    father_list = []

    for t in test["Father"] :
        x = str(t)
        if x[-2:] == "ΗΣ" or x[-2:] == "ΑΣ" :
            x = x[:-1]
        elif x[-2:] == "ΟΣ":
            x = x[:-1] + "Y"
    #     print(x)
        father_list.append(x)        

    test["Father"] = father_list

    test


In [18]:
# 4) Πόλεις

def cities() :    
    
    city_list = []

    for t in test["City"] :
        x = str(t)

        for city1, city2 in city_dict.items() :
            if x == city1:
                x = city2
        
        if x[-1] == "Α" or x[-1] == "Η" :
            x = x + "Σ"
        
        elif x[-2:] == "ΑΣ" :
            x = x[:-1]    
    
        elif x[-2:] == "ΕΣ" or x[-2:] == "ΟΙ":
            x = x[:-2] + "ΩΝ"    
            
        elif x[-1] == "Ι"  :
            x = x + "ΟΥ"

        elif x[-1] == "Ο"  :
            x = x + "Υ"

        elif x[-2:] == "ΟΣ" :
            x = x[:-1] + "Y"

        city_list.append(x)

    test["City"] = city_list

In [19]:
#   5) Λογαριασμός

def account() :
    stoixeia['Account Date'] = pd.to_datetime(stoixeia['ΗΜΕΡ.ΤΕΛΙΚΟΥ ΛΟΓΙΑΡΙΑΣΜΟΥ']).dt.strftime('%d-%m-%Y')
    test['Νumber/date account'] = stoixeia['ΑΡ.ΤΕΛΙΚΟΣ ΛΟΓΑΡΙΑΣΜΟΥ'].astype('string') + "/" + stoixeia['Account Date'].astype('string')
   




In [20]:
# 6) Αίτηση - Define filtered dataframes  

cols = ['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ', 'ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']

loc_date_full = stoixeia.loc[stoixeia[['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ']].notnull().all(1)]
date_full = stoixeia[['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ']].notnull().all(1)

loc_number_full = stoixeia.loc[stoixeia[['ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']].notnull().all(1)]
number_full = stoixeia[['ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']].notnull().all(1)

loc_date_na = stoixeia.loc[stoixeia[['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ']].isnull().all(1)]
date_na = stoixeia[['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ']].isnull().all(1)

loc_number_na = stoixeia.loc[stoixeia[['ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']].isnull().all(1)]
number_na = stoixeia[['ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']].isnull().all(1)   


# Μόνο ημερομηνία

date_full_number_na = stoixeia.loc[(date_na)&(number_na)]

# Μόνο αριθμός λογαριασμού

number_full_date_na = stoixeia.loc[(number_full)&(date_na)]

# Γεμάτα
stoixeia[stoixeia[cols].notnull().all(1)]

# Κενά
stoixeia[stoixeia[cols].isna().all(1)] # Find both nulls columns application

,ΕΠΙΘΕΤΟ,ΟΝΟΜΑ,ΠΑΤΡΩΝΥΜΟ,ΠΟΛΗ,ΟΔΟΣ,ΑΡΙΘΜΟΣ,ΤΚ,ΑΦΜ,ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ,ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ,...,ΗΜΕΡ.ΤΕΛΙΚΟΥ ΛΟΓΙΑΡΙΑΣΜΟΥ,ΠΟΣΟ ΤΕΛΙΚΟΥ ΛΟΓΑΡΙΑΣΜΟΥ,ΗΜΕΡ.ΛΟΓΑΡΙΑΣΜΟΥ,ΠΟΣΟ ΟΦΕΙΛΗΣ,ΑΘΡΟΙΣΜΑ ΠΟΣΟΥ ΟΦΕΙΛΗΣ (2 ΠΑΡΟΧΕΣ),ΣΥΜΒΑΣΗ,ΠΟΛΗ 2,ΟΔΟΣ 2,ΑΡΙΘΜΟΣ 2,ΤΚ 2
26,ΤΖΙΝΙΑ,ΑΙΚΑΤΕΡΙΝΗ,ΑΝΤΩΝΙΟΣ,ΜΟΣΧΑΤΟ,ΟΚΛΑ Θ.,NaN,18233.0,133167667,NaN,NaN,...,2022-02-04 00:00:00,706.84,2022-02-04,706.84,771.34,VOLTON,NaN,NaN,NaN,NaN
36,MEKONNEN,ATSEDE,ZERTHUN,ΑΛΕΞΑΝΔΡΟΥΠΟΛΗ,ΒΑΣΙΛΕΩΣ ΓΕΩΡΓΙΟΥ,10,68400.0,143158389,NaN,NaN,...,2022-01-20 00:00:00,699.59,2022-09-14,691.23,760.91,ΝΟVA,NaN,NaN,NaN,NaN
37,MEKONNEN,ATSEDE,ZERTHUN,ΑΛΕΞΑΝΔΡΟΥΠΟΛΗ,ΒΑΣΙΛΕΩΣ ΓΕΩΡΓΙΟΥ,10,68400.0,143158389,NaN,NaN,...,2020-03-05 00:00:00,72.67,2020-10-01,69.68,760.91,NOVA,NaN,NaN,NaN,NaN
64,ΑΡΑΠΑΚΗ,ΣΠΗΛΙΟ,Έλλειψη πατρωνύμου,ΑΚΡΑΤΑ,ΑΚΡΑΤΑ,NaN,25006.0,110039187,NaN,NaN,...,2022-01-12 00:00:00,761.51,2022-04-13,661.06,NaN,VOLTON,NaN,NaN,NaN,NaN
182,ΔΟΥΚΑ,ΔΗΜΗΤΡΙΟ,ΣΤΥΛΙΑΝΟΣ,ΓΛΥΦΑΔΑ,ΒΥΖΑΝΤΙΟΥ,7,16562.0,104352189,NaN,NaN,...,2022-01-20 00:00:00,1066.50,2022-09-29,320.00,NaN,NOVA,NaN,NaN,NaN,NaN
192,ΑΣΛΑΝΙΔΟΥ,ΙΩΑΝΝΑ,ΒΑΣΙΛΕΙΟΣ,ΞΑΝΘΗ,ΚΩΝΣΤΑΝΤΙΝΟΥ ΒΑΣΙΛΕΩΣ,76,67100.0,53344418,NaN,NaN,...,2022-01-20 00:00:00,542.16,2022-04-07,532.32,NaN,VOLTON,NaN,NaN,NaN,NaN
200,ΤΖΟΛΑ,ΠΑΝΑΓΙΩΤΗ,ΗΛΙΑΣ,ΒΡΑΧΝΑΙΙΚΑ,ΘΕΟΤΟΚΟΠΟΥΛΟΥ,20,26500.0,166645035,NaN,NaN,...,2022-01-27 00:00:00,528.39,2022-09-14,478.00,NaN,VOLTON,NaN,NaN,NaN,NaN
223,ΚΑΡΑΚΙΤΣΟΥ,ΘΩΜΑΙ,ΙΩΑΝΝΗΣ,ΓΛΥΦΑΔΑ,ΑΓΓΕΛΟΥ ΜΕΤΑΞΑ,60,16674.0,131214118,NaN,NaN,...,2022-01-20 00:00:00,541.51,2022-04-07,509.30,NaN,VOLTON,NaN,NaN,NaN,NaN
282,ΔΗΜΟΠΟΥΛΟ,ΔΗΜΗΤΡΙΟ,ΝΙΚΗΤΑΣ,ΚΑΛΑΜΑΤΑ,ΚΑΡΕΛΙΑ ΕΥΣΤΑΘΙΟΥ,49,24100.0,73335394,NaN,NaN,...,2022-01-20 00:00:00,637.22,2022-04-13,467.54,NaN,NOVA,NaN,NaN,NaN,NaN
296,ΙΩΑΝΝΙΔΗ,ΝΙΚΟΛΑΟ,ΙΩΑΝΝΗΣ,ΠΕΙΡΑΙΑΣ,ΑΝΔΡΟΥΤΣΟΥ,154,18535.0,70261070,NaN,NaN,...,2020-12-20 00:00:00,366.89,2021-04-05,312.61,NaN,VOLTON,NaN,NaN,NaN,NaN


In [119]:
# 8) Split παροχές

dict_paroxoi = {} # 1) Για κάθε ξεχωριστό πάροχο δημιούργησε το φιλτραρισμένο df και εισήγαγέ το στο λεξικό παρόχων
   
filtered_df_dict = {}

for paroxos in stoixeia['ΣΥΜΒΑΣΗ '].unique() :
    dict_paroxoi[paroxos] = stoixeia[stoixeia.iloc[:,18]  == paroxos] 


# 2 VOLTON DF Testing Count ΑΦΜs
voldf = dict_paroxoi['VOLTON']
vac = [c for c in voldf['ΑΦΜ '].value_counts()]
voldf['Pool'] = vac





# for dict_paroxoi[paroxos] in dict_paroxoi :
#     dict_paroxoi[paroxos]['Pool']  = dict_paroxoi[paroxos]['ΑΦΜ '].value_counts()                        #    2) Για κάθε df παρόχου στο λεξικο παρόχων εφάρμοσε το countif στο ΑΦΜ -> volton(pool1,2,3), nova(pool1,4) 


# dict_paroxoi['VOLTON']['Pool']  = dict_paroxoi[paroxos]['ΑΦΜ '].value_counts()       




#     λίστα_pool_παροχών = sorted(list(dict_paroxoi[paroxos][['ΑΦΜ ']].value_counts().unique()))           #❌ 3) Για κάθε df παρόχου δημιούργησε λίστες με τον αριθμό countif 
       
#     for pool in λίστα_pool_παροχών :                                                     
#         filtered_df_dict[pool] = dict_paroxoi[paroxos][dict_paroxoi[paroxos][['Pool']]==pool]            #   4) Για κάθε πάροοχο_df, itereate over αριθμό countif στην αντίστοιχη λίστα μοναδικών αριθμών πλήθους παροχών, δημιούργησε το αντίστοιχο filtered df και σώσε το στο κεντρικό filtered_dict 

371

In [ ]:
# Final) Create the dataframe
# # names()
if {'ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ', 'ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ','ΑΡ.ΤΕΛΙΚΟΣ ΛΟΓΑΡΙΑΣΜΟΥ','ΗΜΕΡ.ΤΕΛΙΚΟΥ ΛΟΓΙΑΡΙΑΣΜΟΥ'}.issubset(stoixeia.columns):
    print("Συγχαρητήρια είναι ένα υγιέστατο εξώδικο!")
    account()
    
elif len(stoixeia.columns) > 4 :
    cities()
    test["stoixeia_full"] = test["Ton"] + " " + test["Surname"] + " " + test["Name"] + " " + "του " + test['Father'] + ", κάτοικο "+ test["City"] + ", οδός " + test["Address"] + " αρ. " + test["Number"] + ", ΤΚ " + test["TK"] + ", με ΑΦΜ " + test["ΑΦΜ"] + "."



test["stoixeia"]=  test["Ton"] + " " + test["Surname"] + " " + test["Name"] 
# filename = file[60:80]
# test.to_excel(f"{filename}.xlsx", index = False)

# os.startfile(f"{filename}.xslx")


Συγχαρητήρια είναι ένα υγιέστατο εξώδικο!


In [23]:
# Παράδειγμα για split
# 5) κάνε το split by (pool) rows
(par1)(z)cut =  pool(z).iloc[::(z), :] #1 paroxi Volton_pool_
(par1)(x)cut =  pool(x).iloc[::(x), :] #2 paroxi Volton_pool_
(par1)(y)_cut = pool(y).iloc[::(y), :]   #3 paroxi Volton_pool_


(par2)(z)cut =  pool(z).iloc[::(z), :] #1 paroxi NOVA pool
(par2)(x)cut =  pool(x).iloc[::(x), :] #2 paroxi NOVA pool
(par2)(y)cut = pool(y).iloc[::(y), :]   #3 paroxi NOVA pool_


SyntaxError: invalid syntax (3846270713.py, line 3)